In [1]:
import polars as pl
import re
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def clean_text(text: str) -> str:
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    return text.lower().strip()

In [13]:
df = pl.read_parquet("data/projevy.parquet").filter(
    pl.col("komora_komplet").str.contains('PČR, PS 2021-')
).with_columns(
    pl.col('text').str.len_bytes().alias('delka_projevu')
).filter(
    pl.col('delka_projevu') > 500
)

In [15]:
df.group_by('mluvci').len().sort(by='len',descending=True)

mluvci,len
str,u32
null,8969
"""Místopředseda PSP Jan Bartošek""",1045
"""Místopředsedkyně PSP Olga Rich…",1001
"""Místopředseda PSP Jan Skopeček""",900
"""Předsedkyně PSP Markéta Pekaro…",862
…,…
"""Hlasování číslo 261""",1
"""Hlasování číslo 302""",1
"""Hlasování číslo 287""",1


In [33]:
mluvci_df = (
    df.filter(pl.col("mluvci").is_not_null())
    .with_columns(pl.col("text").map_elements(clean_text))
    .group_by("mluvci")
    .agg(pl.col("text").str.concat(" ").alias("mluvci_text"))
    .drop_nulls()
)

C:\Users\micha\AppData\Local\Temp\ipykernel_39292\3577411899.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(pl.col("text").map_elements(clean_text))
C:\Users\micha\AppData\Local\Temp\ipykernel_39292\3577411899.py:5: DeprecationWarning: `str.concat` is deprecated; use `str.join` instead. Note also that the default `delimiter` for `str.join` is an empty string, not a hyphen.
  .agg(pl.col("text").str.concat(" ").alias("mluvci_text"))


In [34]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Generate embeddings
mluvci_texts = chamber_df["mluvci_text"].to_list()
mluvci_embeddings = model.encode(mluvci_texts)

# Add embeddings to DataFrame
mluvci_df = mluvci_df.with_columns(
    pl.Series(name="embedding", values=mluvci_embeddings.tolist())
)


In [36]:
mluvci_df

mluvci,mluvci_text,embedding
str,str,list[f64]
"""Hlasování s pořadovým číslem 1…","""přihlášeno je 139 poslankyň a …","[-0.122616, 0.120714, … 0.054628]"
"""Hlasování číslo 371""","""přihlášeno 168 poslanců a posl…","[0.077109, 0.265332, … 0.112575]"
"""Místopředsedkyně PSP Klára Dos…","""děkuji moc za slovo paní předs…","[0.105253, 0.06577, … 0.173168]"
"""Hlasování s pořadovým číslem 2…","""přihlášeno je 183 poslankyň a …","[0.147597, 0.048086, … 0.126322]"
"""Hlasování číslo 87""","""bylo přihlášeno 144 poslanců a…","[0.019895, 0.33498, … 0.132347]"
…,…,…
"""Hlasování číslo 71""","""přihlášeno 155 poslanců a posl…","[-0.137887, 0.164969, … 0.201891]"
"""Poslanec Marek Ženíšek""","""děkuji za slovo paní předsedaj…","[-0.006633, 0.227894, … 0.173489]"
"""Hlasování číslo 70""","""přihlášeno 119 poslankyň a pos…","[-0.0176, 0.067547, … 0.124948]"


In [22]:
import numpy as np

In [31]:
emb_matrix = np.array(mluvci_df["embedding"].to_list())

# Reduce dimensionality with PCA
pca = PCA(n_components=2)
reduced_emb = pca.fit_transform(emb_matrix)

# Add to DataFrame
mluvci_df = mluvci_df.with_columns(
    pl.Series(name="x", values=reduced_emb[:, 0]),
    pl.Series(name="y", values=reduced_emb[:, 1])
)

ColumnNotFoundError: "embedding" not found

In [29]:
mluvci_df

mluvci,mluvci_text,x,y
str,str,f64,f64
"""Poslanec Petr Beitl""","""vážený pane předsedající vážen…",1.316167,1.326166
"""Hlasování číslo 108""","""přihlášeno 163 poslanců pro 91…",-0.013208,0.310584
"""Hlasování číslo 84""","""byli přihlášeni 144 přítomní p…",-0.698717,-0.177151
"""Hlasování s pořadovým číslem 7…","""přihlášeno je 156 poslankyň a …",-0.882092,-0.318835
"""Hlasování s pořadovým číslem 2…","""přihlášeno je 166 poslankyň a …",-0.797364,0.05775
…,…,…,…
"""Poslanec Jiří Havránek""","""vážený pane místopředsedo já v…",0.689627,-0.525456
"""Hlasování číslo 138""","""přihlášeno 160 poslankyň a pos…",-0.825558,0.88512
"""Hlasování číslo 93""","""přihlášeno 167 poslanců a posl…",-0.580592,0.167661


In [27]:
plt.figure(figsize=(12, 8))
ax = plt.subplot()

# Plot each chamber
for row in mluvci_df.iter_rows(named=True):
    ax.scatter(row["x"], row["y"], s=150)
    ax.annotate(
        row["mluvci"], 
        (row["x"], row["y"]),
        xytext=(5, 2),
        textcoords='offset points',
        fontsize=9,
        alpha=0.8
    )

# Add plot decorations
plt.title("Mluvčí Similarity Based on Text Embeddings", fontsize=14)
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.grid(alpha=0.2)
plt.tight_layout()

# Add distance indicators (optional)
for i in range(len(reduced_emb)):
    for j in range(i+1, len(reduced_emb)):
        plt.plot(
            [reduced_emb[i,0], reduced_emb[j,0]],
            [reduced_emb[i,1], reduced_emb[j,1]],
            'gray', 
            linewidth=0.5, 
            alpha=0.3
        )

plt.show()


KeyboardInterrupt

